In [ ]:
import numpy as np
import pathlib
import os
import re
import math
import imageio
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import cv2  

In [ ]:
# Set the folder path where images are located and the tile size
path = pathlib.Path(os.getcwd())
path_gt = path/'Hagen_testmix/gt_images'
path_noisy = path/'Hagen_testmix/noisy_images'
path_Base = path/'Hagen_testmix/Hagen_pred_images'
path_Transfer = path/'Hagen_testmix/Transfer_pred_images'
path_Transfer_all = path/'Hagen_testmix/Transfer_all_pred_images'
samples = ['actin-20x-noise1','actin-60x-noise1','actin-60x-noise2','mito-20x-noise1','mito-60x-noise1','mito-60x-noise2',
           'actin-confocal','mito-confocal','nucleus','membrane']

In [ ]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
# scale and shift as described by Weigert et al
def scale_and_shift(image1,image2):
    
    y = np.array(image2/(2**16-1))        # gt, already percentile normalized in image_to_tiles
    y_hat = np.array(image1/(2**16-1))
    
    def mse_fn(params):
        a, b = params
        y_hat_scaled = a*y_hat+b
        mse = np.mean((y-y_hat_scaled)**2)
        return mse

    initial_guess = [1.0, 0.0]
    boundaries = [(-20,20),(-1,1)]

    result = minimize(mse_fn,initial_guess,bounds=boundaries)

    if result.success:
        optimized_a,optimized_b = result.x
        y_hat_scaled = optimized_a*y_hat+optimized_b
        return optimized_a,optimized_b,y_hat_scaled
    else:
        raise Exception("optimization failed:" + result.message)

In [ ]:
# applying scale and shift as in the paper by Hagen et al.
def calculate_ssim_and_psnr(image1_path, image2_path, scale=True):
    # Read the two images
    image1 = imageio.v2.imread(image1_path)
    image2 = imageio.v2.imread(image2_path)

    # Convert images to grayscale (if they are not already)
    if len(image1.shape) == 3:
        image1 = image1[:,:,0]
    if len(image2.shape) == 3:
        image2 = image2[:,:,0]

    if scale:    # scale and shift according to Weigert et al and Hagen et al
        a,b,image1 = scale_and_shift(image1,image2)
        image2 = image2/(2**16-1)
    else:
        image1 = image1/(2**16-1)
        image2 = image2/(2**16-1)
    
    # Calculate SSIM
    ssim_value = ssim(image1, image2, data_range=1)   
    
    # Calculate PSNR
    def calculate_psnr(img1, img2):
        # img1 and img2 have range [0, 1] after normalization
        img1 = img1.astype(np.float64)
        img2 = img2.astype(np.float64)
        mse = np.mean((img1 - img2)**2)
        if mse == 0:
            return float('inf')
        return 10 * math.log10(1 / mse)                        # like in Hagen et al           
    
    psnr_value = calculate_psnr(image1, image2)

    return ssim_value, psnr_value

In [ ]:
# get metrics for original noise images - derived as published
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_noisy/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

In [ ]:
# get metrics for Base model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Base/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

In [ ]:
# get metrics for Transfer model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Transfer/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

In [ ]:
# get metrics for Transfer model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Transfer_all/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []